## Adversarial Attack on Neral Network

Introduction:



Small scale implement on the "toy model":

generate toy model:

In [ ]:
import pandas as pd
import pickle
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import losses
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method


In [2]:
df = pd.read_csv('../data/test_train.zip').drop(columns='Unnamed: 0')
df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_is_S2,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH,normal,test
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,310,1881,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
494017,0,282,2286,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
494018,0,203,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
494019,0,291,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0


In [3]:
np.random.seed(df.shape[0])


In [4]:
# make sure the toy dataset includes every class
df0 = df[df.test == 0]
df1 = df[df.test == 1]
df2 = df[df.test == 2]
df3 = df[df.test == 3]
df4 = df[df.test == 4]

In [5]:
print(df0.shape[0]/50)
print(df1.shape[0]/50)
print(df2.shape[0]/50)
print(df3.shape[0]/50)
print(df4.shape[0]/50)

1945.56
7829.16
22.52
1.04
82.14


In [6]:
df00 = df0.sample(n=1945)
df01 = df1.sample(n=7829)
df02 = df2.sample(n=22)
df03 = df3.sample(n=5)
df04 = df4.sample(n=82)


In [7]:
dfs = pd.concat([df00,df01,df02,df03,df04])
dfs = dfs.sample(frac=1)
dfs

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_is_S2,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH,normal,test
156238,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
210735,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
438205,0,520,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
240342,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
301512,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128892,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
24843,0,240,10378,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
424130,0,520,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
252542,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0


In [8]:
dfs

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_is_S2,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH,normal,test
156238,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
210735,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
438205,0,520,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
240342,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
301512,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128892,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
24843,0,240,10378,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
424130,0,520,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0
252542,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,False,1.0


In [9]:
dfs.test.unique()

array([1., 0., 4., 3., 2.])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    dfs.iloc[:,:-2], dfs.iloc[:,-1], test_size=0.3, random_state=42)

In [11]:
y_test

179242    1.0
469646    1.0
158054    1.0
136854    0.0
444891    1.0
         ... 
148692    0.0
148827    0.0
108470    1.0
291450    1.0
273968    1.0
Name: test, Length: 2965, dtype: float64

In [12]:
# build a simple neural network model  
model0 = Sequential()
model0.add(Dense(50, input_dim=118, activation='relu'))
model0.add(Dense(50, activation='relu'))
model0.add(Dense(5, activation='softmax'))
model0.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model0.fit(X_train, y_train, epochs=5, batch_size=128)


Epoch 1/5
55/55 [==============================] - 2s 7ms/step - loss: 96.0801 - accuracy: 0.4652
Epoch 2/5
55/55 [==============================] - 0s 6ms/step - loss: 25.9924 - accuracy: 0.9306
Epoch 3/5
55/55 [==============================] - 0s 8ms/step - loss: 15.0021 - accuracy: 0.9532
Epoch 4/5
55/55 [==============================] - 0s 4ms/step - loss: 4.8580 - accuracy: 0.9682
Epoch 5/5
55/55 [==============================] - 0s 7ms/step - loss: 12.2163 - accuracy: 0.9786


In [27]:
model0.evaluate(X_train,y_train)

217/217 [==============================] - 2s 4ms/step - loss: 9.7054 - accuracy: 0.9718


[9.70541763305664, 0.9718126654624939]

In [28]:
model0.evaluate(X_test,y_test)

93/93 [==============================] - 1s 5ms/step - loss: 1.1620 - accuracy: 0.9707


[1.1620385646820068, 0.9706576466560364]

The origianl model has achieve 97% of accuracy on both train set and test set. Now try to add noise to the test set:

In [29]:
random_noise = []
describe = dfs.describe()
for i in range(df.shape[1]-2):
    mean = describe.iloc[1,i]
    std = describe.iloc[2,i]
    random_noise.append(np.random.normal(mean, std)/1000)
random_noise = np.array(random_noise)
random_noise

array([-4.19246854e-01,  8.05345611e+00,  1.45346728e+00,  0.00000000e+00,
       -3.03283513e-04,  2.35612534e-05,  4.54506484e-04,  1.15419675e-05,
        3.27279468e-04, -3.40462153e-05, -8.59867177e-06,  0.00000000e+00,
        4.47604419e-06, -1.61383647e-05,  6.17529791e-06, -4.81550964e-06,
        0.00000000e+00,  0.00000000e+00, -5.61639352e-06,  1.06418465e-01,
        3.17215041e-01,  1.86734931e-05,  4.57496588e-04,  1.50962150e-04,
        1.22695582e-04,  2.49862360e-04,  1.91371863e-05, -2.47475105e-04,
        2.38483935e-01, -6.73886800e-03,  7.83117357e-04, -1.23973225e-04,
        5.83708395e-04, -2.44991575e-05, -3.10005089e-05, -5.74771784e-06,
       -3.07981968e-04,  3.13511561e-04,  1.06020070e-04, -1.40683714e-04,
       -1.73230673e-04, -2.62047427e-04,  9.75500317e-05,  8.00129283e-06,
       -5.91882449e-06, -3.32524083e-05,  5.79258323e-05,  2.01203322e-05,
       -2.94477306e-05, -7.00646685e-05,  1.23848512e-03,  7.37158388e-05,
        7.91095103e-04,  

In [30]:
X_noise=[]
for i in range(X_test.shape[0]):
    X_noise.append(np.array(X_test.iloc[i])+random_noise)
X_noise_test = pd.DataFrame(X_noise, columns = X_test.columns)

In [31]:
X_noise_test

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_is_S1,flag_is_REJ,flag_is_S2,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH
0,-0.419247,1040.053456,1.453467,0.0,-0.000303,0.000024,0.000455,0.000012,0.000327,-0.000034,...,-0.000003,0.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
1,-0.419247,8.053456,1.453467,0.0,-0.000303,0.000024,0.000455,0.000012,0.000327,-0.000034,...,-0.000003,1.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
2,-0.419247,1040.053456,1.453467,0.0,-0.000303,0.000024,0.000455,0.000012,0.000327,-0.000034,...,-0.000003,0.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
3,-0.419247,198.053456,27739.453467,0.0,-0.000303,0.000024,0.000455,0.000012,1.000327,-0.000034,...,-0.000003,0.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
4,-0.419247,528.053456,1.453467,0.0,-0.000303,0.000024,0.000455,0.000012,0.000327,-0.000034,...,-0.000003,0.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2960,0.580753,640.053456,333.453467,0.0,-0.000303,0.000024,0.000455,0.000012,1.000327,-0.000034,...,-0.000003,0.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
2961,-0.419247,8.053456,1.453467,0.0,-0.000303,0.000024,0.000455,0.000012,0.000327,-0.000034,...,-0.000003,1.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
2962,-0.419247,8.053456,1.453467,0.0,-0.000303,0.000024,0.000455,0.000012,0.000327,-0.000034,...,-0.000003,0.000086,0.000005,1.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011
2963,-0.419247,1040.053456,1.453467,0.0,-0.000303,0.000024,0.000455,0.000012,0.000327,-0.000034,...,-0.000003,0.000086,0.000005,0.00001,0.000006,0.000008,-0.000056,0.0,0.0,-0.000011


In [32]:
model0.evaluate(X_noise_test,y_test)

93/93 [==============================] - 0s 5ms/step - loss: 1.1733 - accuracy: 0.9717


[1.173290491104126, 0.9716694951057434]

After adding random noise to the test data set, the neural network model performs a 97% accuracy. \
This means random noise does not decrease the performance of the model.

Now try to attack the model under different parameter epsilon:

In [68]:
print('On train set: ')
for i in range(1,-4,-1):
    
    logits_model = tf.keras.Model(model0.input, model0.layers[-1].output)

    epsilon = 10**i
    print('epsilon is '+str(epsilon))
    adv_fgsm_x = fast_gradient_method(logits_model,
                                      X_train,
                                      epsilon,
                                      np.inf,
                                      targeted=False)

    adv_x = adv_fgsm_x
    print(model0.evaluate(adv_x,y_train))


On train set: 
epsilon is 10
217/217 [==============================] - 0s 773us/step - loss: 114.1478 - accuracy: 0.1352
[114.14776611328125, 0.13515466451644897]
epsilon is 1
217/217 [==============================] - 0s 759us/step - loss: 11.5101 - accuracy: 0.8154
[11.51010513305664, 0.8154090642929077]
epsilon is 0.1
217/217 [==============================] - 0s 783us/step - loss: 9.6878 - accuracy: 0.9714
[9.687848091125488, 0.9713789820671082]
epsilon is 0.01
217/217 [==============================] - 0s 778us/step - loss: 9.7031 - accuracy: 0.9717
[9.703137397766113, 0.9716681241989136]
epsilon is 0.001
217/217 [==============================] - 0s 917us/step - loss: 9.7052 - accuracy: 0.9718
[9.705185890197754, 0.9718126654624939]


In [65]:
print('On test set: ')
for i in range(1,-4,-1):
    
    logits_model = tf.keras.Model(model0.input, model0.layers[-1].output)

    epsilon = 10**i
    print('epsilon is '+str(epsilon))
    adv_fgsm_x = fast_gradient_method(logits_model,
                                      X_test,
                                      epsilon,
                                      np.inf,
                                      targeted=False)

    adv_x = adv_fgsm_x
    print(model0.evaluate(adv_fgsm_x,y_test))


On test set: 
epsilon is 10
93/93 [==============================] - 0s 870us/step - loss: 101.1066 - accuracy: 0.1558
[101.10657501220703, 0.1558178812265396]
epsilon is 1
93/93 [==============================] - 0s 968us/step - loss: 2.8576 - accuracy: 0.8159
[2.8576033115386963, 0.815851628780365]
epsilon is 0.1
93/93 [==============================] - 0s 935us/step - loss: 1.1359 - accuracy: 0.9693
[1.135924220085144, 0.969308614730835]
epsilon is 0.01
93/93 [==============================] - 0s 902us/step - loss: 1.1588 - accuracy: 0.9703
[1.158756136894226, 0.9703204035758972]
epsilon is 0.001
93/93 [==============================] - 0s 968us/step - loss: 1.1617 - accuracy: 0.9707
[1.1617047786712646, 0.9706576466560364]


choose epsilon=1, generate adversarial examples and see how the orginal model would perform:

In [98]:
logits_model = tf.keras.Model(model0.input, model0.layers[-1].output)

epsilon = 1
print('epsilon is '+str(epsilon))
adv_fgsm_x = fast_gradient_method(logits_model,
                                  X_train,
                                  epsilon,
                                  np.inf,
                                  targeted=False)

adv_x0 = adv_fgsm_x
print(model0.evaluate(adv_x0,y_train))


epsilon is 1
217/217 [==============================] - 0s 764us/step - loss: 11.5101 - accuracy: 0.8154
[11.51010513305664, 0.8154090642929077]


In [127]:
logits_model = tf.keras.Model(model0.input, model0.layers[-1].output)

epsilon = 1
print('epsilon is '+str(epsilon))
adv_fgsm_x = fast_gradient_method(logits_model,
                                  X_test,
                                  epsilon,
                                  np.inf,
                                  targeted=False)

adv_x0 = adv_fgsm_x
print(model0.evaluate(adv_x0,y_test))


epsilon is 1
93/93 [==============================] - 0s 913us/step - loss: 2.8576 - accuracy: 0.8159
[2.8576033115386963, 0.815851628780365]


After adversarial attack, the accuracy of the model decreased to 81%

Take the adversarial example as input and re-train the neural network:

In [118]:
# input inclues the orginal train data and the adversarial examples that we generate based on them.
adv_train = tf.concat([tf.constant(X_train.astype('float32')),adv_x0],0)

In [119]:
model01 = Sequential()
model01.add(Dense(50, input_dim=118, activation='relu'))
model01.add(Dense(50, activation='relu'))
model01.add(Dense(5, activation='softmax'))
model01.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model01.fit(adv_train, pd.concat([y_train,y_train]), epochs=5, batch_size=128)


Epoch 1/5
109/109 [==============================] - 0s 880us/step - loss: 39.0608 - accuracy: 0.8803
Epoch 2/5
109/109 [==============================] - 0s 834us/step - loss: 18.6903 - accuracy: 0.9739
Epoch 3/5
109/109 [==============================] - 0s 880us/step - loss: 25.3574 - accuracy: 0.9746
Epoch 4/5
109/109 [==============================] - 0s 926us/step - loss: 20.8956 - accuracy: 0.9795
Epoch 5/5
109/109 [==============================] - 0s 898us/step - loss: 39.8348 - accuracy: 0.9738


In [126]:
logits_model = tf.keras.Model(model01.input, model01.layers[-1].output)

epsilon = 1
print('epsilon is '+str(epsilon))
adv_fgsm_x = fast_gradient_method(logits_model,
                                  adv_train,
                                  epsilon,
                                  np.inf,
                                  targeted=False)

adv_x1 = adv_fgsm_x
print(model01.evaluate(adv_x1,pd.concat([y_train,y_train])))


epsilon is 1
433/433 [==============================] - 0s 521us/step - loss: 22.1525 - accuracy: 0.8807
[22.152494430541992, 0.8807458877563477]


After adversarial training, under the same adversatial attack the accuracy decreased to 88%. Comparing to the accuracy before adversarial training(81%), the adversarial trianing has defended the attack partly.

In [129]:
# result = model01.evaluate(adv_x1,pd.concat([y_train,y_train]))
# dict(zip(model01.metrics_names, result))

433/433 [==============================] - 0s 516us/step - loss: 22.1525 - accuracy: 0.8807


{'loss': 22.152494430541992, 'accuracy': 0.8807458877563477}

Import the models Adam built (see report 0-)

In [137]:
test = pd.read_csv("../data/processed/test.csv").drop(columns = ['Unnamed: 0'])
train = pd.read_csv("../data/processed/train.csv").drop(columns = ['Unnamed: 0'])

In [152]:
X_train = train.drop('normal',axis=1)
X_test = train.drop('normal',axis=1)
y_train = train['normal']
y_test = train['normal']

In [134]:
# model1 = keras.models.load_model("../data/processed/models/model1") 

In [ ]:
for i in range(1,12):
    

In [23]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
# from art.utils import 

In [29]:
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()


In [33]:
# Step 3: Create the ART classifier
classifier = KerasClassifier(model=model0, clip_values=(mins,maxs), use_logits=False)


ValueError: Invalid `clip_values`: min >= max.

In [ ]:
classifier.fit(X_train, y_train, batch_size=64, nb_epochs=3)

In [24]:
describe

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_is_REJ,flag_is_S2,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH,test
count,9883.000000,9.883000e+03,9883.000000,9883.0,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,...,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.0,9883.0,9883.000000,9883.000000
mean,51.986340,1.582295e+03,557.398462,0.0,0.007791,0.000101,0.031671,0.000202,0.149449,0.005869,...,0.054235,0.000101,0.177072,0.000101,0.001214,0.002024,0.0,0.0,0.000101,0.831327
std,791.244348,5.625788e+04,2998.009261,0.0,0.151362,0.010059,0.719456,0.014225,0.356548,0.179291,...,0.226491,0.010059,0.381749,0.010059,0.034826,0.044942,0.0,0.0,0.010059,0.497293
min,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
25%,0.000000,4.300000e+01,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000
50%,0.000000,5.200000e+02,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000
75%,0.000000,1.032000e+03,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000
max,30317.000000,5.133876e+06,64786.000000,0.0,3.000000,1.000000,28.000000,1.000000,1.000000,16.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,4.000000


In [26]:
min_max = []
for feature in describe:
    min_max.append((describe.loc['min',feature],describe.loc['max',feature]))

In [28]:
min_max=np.array(min_max)

In [32]:
mins = []
maxs = []
for feature in describe:
    mins.append(describe.loc['min',feature])
    maxs.append(describe.loc['max',feature])
mins = np.array(mins)
maxs = np.array(maxs)

In [240]:
model = model0
fmodel = model0

In [233]:
import tensorflow as tf
import eagerpy as ep
from foolbox import TensorFlowModel, accuracy, samples, Model
from foolbox.attacks import LinfPGD

In [238]:
# instantiate a model (could also be a TensorFlow or JAX model)
model = tf.keras.applications.ResNet50(weights="imagenet")
pre = dict(flip_axis=-1, mean=[104.0, 116.0, 123.0])  # RGB to BGR
# fmodel: Model = TensorFlowModel(model, bounds=(0, 255), preprocessing=pre)
# fmodel = fmodel.transform_bounds((0, 1))


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [287]:
# get data and test the model
# wrapping the tensors with ep.astensors is optional, but it allows
# us to work with EagerPy tensors in the following
# images, labels = ep.astensors(*samples(fmodel, dataset="imagenet", batchsize=16))
# clean_acc = accuracy(fmodel, images, labels)
# print(f"clean accuracy:  {clean_acc * 100:.1f} %")



In [274]:
# apply the attack
attack = LinfPGD()
epsilons = [
    0.0,
    0.0002,
    0.0005,
    0.0008,
    0.001,
    0.0015,
    0.002,
    0.003,
    0.01,
    0.1,
    0.3,
    0.5,
    1.0,
]



In [309]:
# fmodel: Model = TensorFlowModel(model0, bounds=(0,1))
# fmodel = fmodel.transform_bounds((0, 1))
# fmodel = KerasModel(model0, bounds=(0,1))
fmodel: Model = TensorFlowModel(model0, bounds=(0,118))
fmodel = fmodel.transform_bounds((0, 118))

images = tf.constant(X_test.iloc[0])
labels = tf.constant(y_test.iloc[0])
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)


ValueError: Exception encountered when calling layer "sequential_48" (type Sequential).

Input 0 of layer "dense_138" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (118,)

Call arguments received:
  • inputs=tf.Tensor(shape=(118,), dtype=float64)
  • training=False
  • mask=None

In [249]:
attack

LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)

In [ ]:
# calculate and report the robust accuracy (the accuracy of the model when
# it is attacked)
robust_accuracy = 1 - success.float32().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

# we can also manually check this
# we will use the clipped advs instead of the raw advs, otherwise
# we would need to check if the perturbation sizes are actually
# within the specified epsilon bound
print()
print("we can also manually check this:")
print()
print("robust accuracy for perturbations with")


In [ ]:
for eps, advs_ in zip(epsilons, clipped_advs):
    acc2 = accuracy(fmodel, advs_, labels)
    print(f"  Linf norm ≤ {eps:<6}: {acc2 * 100:4.1f} %")
    print("    perturbation sizes:")
    perturbation_sizes = (advs_ - images).norms.linf(axis=(1, 2, 3)).numpy()
    print("    ", str(perturbation_sizes).replace("\n", "\n" + "    "))
    if acc2 == 0:
        break
